In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#filepath = 'C:/Users/I008328/Desktop/AIA/Next Recommended Actions/'
filepath = 'D:/Users/I008328/NRA/data/'
plt.style.use('seaborn')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.3f}'.format)

In [2]:
import os
import pickle
import pyodbc
import cx_Oracle
import sqlalchemy
import time
import datetime
import re
from sqlalchemy import create_engine

In [3]:
start = time.time()
 
#oracle_connection_string = 'oracle+cx_oracle://{username}:{password}@{hostname}:{port}/?service_name={database}'
oracle_connection_string = 'oracle+cx_oracle://{username}:{password}@{host}:{port}/?service_name={database}'

engine = create_engine(
    oracle_connection_string.format(
        username='KMDB016',
        password='mktBAtg1688!',
#        hostname='mybfdp1c1-scan.my1.ocm.s7067199.oraclecloudatcustomer.com',
        host = '10.136.107.7',
        #hostname = 'KUDWH01.my2.ocm.s7067200.oraclecloudatcustomer.com',
        port='1521',
        database='KPDWH01.MY1.OCM.S7067199.ORACLECLOUDATCUSTOMER.COM'
    )
    )

In [4]:
# Only Get Individual Customers
sql  = " SELECT * FROM NRA_PRED_SEP_TRAIN_2020 "

df_sep = pd.read_sql(sql, engine)

end = time.time()

print("Time Taken: {:.2f}s".format(end-start))

df_sep.shape

Time Taken: 207.09s


(2420403, 58)

In [5]:
df_sep.to_pickle(filepath + 'Sep2020_Pred.pkl')

In [14]:
sql = " SELECT * FROM INS_NB_4M_OCT_2020 "
df_repurchase = pd.read_sql(sql, engine)
df_repurchase.shape

(149731, 3)

In [16]:
sql = " SELECT * FROM OMKTDM.VW_CUST360_F_CUSTOMER_HIST_M@OUSRMKT_KUODA02 \
        WHERE TRUNC(REF_DT) = TO_DATE('30/09/2020','DD/MM/YYYY') AND INFORCE_IND = 'Y' "
df_sep_flags = pd.read_sql(sql, engine)
df_sep_flags.shape

(3218856, 68)

In [44]:
df_repurchase.to_pickle(filepath + 'Oct_2020_Repurchase.pkl')

In [10]:
df_sep_flags = pd.read_pickle(filepath + 'Sep2020_snapshot.pkl')
#df_sep = pd.read_pickle(filepath + 'Sep2020_Pred.pkl')
df_repurchase = pd.read_pickle(filepath + 'Oct_2020_Repurchase.pkl')

In [52]:
df_sep_flags.to_csv(filepath + 'Sep2020_cust360.csv',index=False)
df_sep.to_csv(filepath + 'Sep2020_Pred.csv',index=False)
df_repurchase.to_csv(filepath + 'Oct_2020_Repurchase.csv',index=False)

In [ ]:
df_sep_flags.to_csv(filepath + 'Sep2020_cust360.csv',index=False)
df_sep.to_csv(filepath + 'Sep2020_Pred.csv',index=False)
df_repurchase.to_csv(filepath + 'Oct_2020_Repurchase.csv',index=False)

In [17]:
df_sep.columns = df_sep.columns.str.upper()
df_repurchase.columns = df_repurchase.columns.str.upper()
df_sep_flags.columns = df_sep_flags.columns.str.upper()

In [18]:
df_sep.drop_duplicates(subset='INS_ID',keep='first',inplace=True, ignore_index=True)
df_sep.shape

(2386161, 58)

In [19]:
df_repurchase.head()

SCV_ID  REPURCHASE_ANP  TARGET_REPUR_IND
0      74            1440                 1
1     161            4685                 1
2    1007            7798                 1
3    1099            1109                 1
4    1122            2477                 1

In [1]:
#  12068654 , 17058350
#df_sep[df_sep.SCV_ID == 17058350]

In [2]:
#df_sep_flags[df_sep_flags.SCV_ID == 17058350]

In [3]:
#df_sep_flags[df_sep_flags.SCV_ID == 3119164]

In [4]:
#df_train = df_sep.merge(df_sep_flags, on=['SCV_ID'],suffixes=('_pred', '_flags'))
#df_train.shape

In [20]:
df_flags = df_sep_flags[['SCV_ID','FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']]
df_flags.head()

SCV_ID FLG_AGE_21 FLG_MARRIED FLG_LIFE_STAGE FLG_CHILD FLG_INCOME_CHANGE FLG_BIRTHDAY FLG_FHC_ANNUAL FLG_CLM_TRIGGER FLG_CLM_FAM_TRIGGER FLG_CALL_TRIGGER FLG_MATURING FLG_MATURED
0  19245881          Y           N              N         N                 N            N              N               N                   N                N            N           N
1   4500794          Y           N              N         N                 N            N              N               N                   N                N            N           N
2  19531414          Y           N              N         N                 N            N              N               N                   N                N            N           N
3  15675919          Y           N              N         N                 N            N              N               N                   N                N            N           N
4  19257208          Y           N              N         N                 N            N              N               N                   N                N            N           N

In [21]:
targets = {'Y':1, 'N':0}
df_flags[['FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']] = \
df_flags_map = df_sep_flags[['FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']].stack().map(targets).unstack()

C:\Users\I008328\.conda\envs\kw-py37\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [22]:
df_flags1 = df_flags.groupby(['SCV_ID'],as_index=False).sum()
df_flags1

SCV_ID  FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0               2           2            0               0          0                  0             0               0                0                    0                 0             0            0
1               3           1            0               1          0                  0             0               0                0                    0                 0             0            0
2               4           1            0               0          0                  0             1               0                0                    0                 0             0            0
3               5           2            0               0          0                  0             0               0                0                    0                 0             0            0
4               6           4            0               0          0                  0             0               0                0                    0                 0             0            0
...           ...         ...          ...             ...        ...                ...           ...             ...              ...                  ...               ...           ...          ...
2617221  20975571           1            0               0          0                  0             0               0                0                    0                 0             0            0
2617222  20975692           1            0               0          0                  0             0               0                0                    0                 0             0            0
2617223  20975734           1            0               0          0                  0             0               0                0                    0                 0             0            0
2617224  20975753           1            0               0          0                  0             0               0                0                    0                 0             0            0
2617225  20975865           1            0               0          0                  0             0               0                0                    0                 0             0            0

[2617226 rows x 13 columns]

In [23]:
# transform back to 0 and 1
df_flags1.iloc[:,1:][df_flags1.iloc[:,1:] > 0] = 1

C:\Users\I008328\.conda\envs\kw-py37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\I008328\.conda\envs\kw-py37\lib\site-packages\pandas\core\frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [24]:
df_flags1['FLG_SUM'] = df_flags1.iloc[:,1:].sum(axis=1)

In [25]:
cols = list(df_flags1.iloc[:,1:-1].columns)
cols

['FLG_AGE_21',
 'FLG_MARRIED',
 'FLG_LIFE_STAGE',
 'FLG_CHILD',
 'FLG_INCOME_CHANGE',
 'FLG_BIRTHDAY',
 'FLG_FHC_ANNUAL',
 'FLG_CLM_TRIGGER',
 'FLG_CLM_FAM_TRIGGER',
 'FLG_CALL_TRIGGER',
 'FLG_MATURING',
 'FLG_MATURED']

In [17]:
df_sep_flags.columns

Index(['REF_DT', 'CUSTOMER_UNIQUE_ID', 'COMMON_IC', 'NEW_IC', 'OLD_IC', 'SERVICING_AGENT_CD', 'SCV_ID', 'CUSTOMER_ROLE', 'INFORCE_IND', 'ANN_INCOME', 'MARITAL_STATUS', 'LIFE_STAGE', 'OCCUPATION', 'NO_OF_CHILD', 'VIT_MEMBERSHIP_STATUS', 'VIT_STATUS_LAST_12M', 'VIT_AMOUNT_DUE', 'VIT_AMOUNT_DUE_DATE', 'VIT_POINTS', 'VIT_STATUS_CURRENT', 'VIT_POITNS_BAL', 'VIT_BAL_DUE_DATE', 'VIT_STATUS_BAL', 'VIT_BAL_STATEMENT_IND', 'FLG_REPURCHASE_YTD', 'REPURCHASE_LIKELIHOOD', 'LAPSE_LIKELIHOOD', 'AD_CONV_LIKELIHOOD', 'FLG_MARRIED_UPDATE_DT', 'FLG_LIFE_STAGE_UPDATE_DT', 'FLG_CHILD_UPDATE_DT', 'FLG_INCOME_UPDATE_DT', 'FLG_POL_ANNIVERSARY', 'FLG_NON_ACUTE_CLM', 'FLG_NON_ACUTE_CLM_AMT', 'FLG_ACUTE_CLM', 'FLG_ACUTE_CLM_AMT', 'FLG_ACC_CLM', 'FLG_ACC_CLM_AMT', 'FLG_NON_ACUTE_CLM_FAM', 'FLG_NON_ACUTE_CLM_AMT_FAM', 'FLG_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_AMT_FAM', 'FLG_ACC_CLM_FAM', 'FLG_ACC_CLM_AMT_FAM', 'FLG_CALL_POSITIVE', 'CALL_DATE', 'CALL_TYPE', 'FLG_CALL_NEGATIVE', 'MYAIA_REGISTRATION_IND',
       'FHC_COMPL

In [26]:
df_sep_flags.drop(cols,axis=1,inplace=True)

In [27]:
df_sep_flags.shape

(3218856, 56)

In [28]:
df_sep_flags.drop_duplicates(subset='SCV_ID',keep='first',inplace=True, ignore_index=True)
df_sep_flags.shape

(2617226, 56)

In [29]:
df_flags1.iloc[:,1:-1]

FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0                 1            0               0          0                  0             0               0                0                    0                 0             0            0
1                 1            0               1          0                  0             0               0                0                    0                 0             0            0
2                 1            0               0          0                  0             1               0                0                    0                 0             0            0
3                 1            0               0          0                  0             0               0                0                    0                 0             0            0
4                 1            0               0          0                  0             0               0                0                    0                 0             0            0
...             ...          ...             ...        ...                ...           ...             ...              ...                  ...               ...           ...          ...
2617221           1            0               0          0                  0             0               0                0                    0                 0             0            0
2617222           1            0               0          0                  0             0               0                0                    0                 0             0            0
2617223           1            0               0          0                  0             0               0                0                    0                 0             0            0
2617224           1            0               0          0                  0             0               0                0                    0                 0             0            0
2617225           1            0               0          0                  0             0               0                0                    0                 0             0            0

[2617226 rows x 12 columns]

In [30]:
# Sanity check to make sure only 0 and 1
df_flags1.iloc[:,1:-1].apply(pd.Series.value_counts).fillna(0)

FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0       0.000      2604532         2487050    2581875            2600704       2377408     2617226.000          2610786              2602228           2582064       2606738      2614730
1 2617226.000        12694          130176      35351              16522        239818           0.000             6440                14998             35162         10488         2496

In [31]:
# Cust360 with flags cleaned
df_train1 = pd.merge(df_sep_flags,df_flags1,how='inner',on='SCV_ID')
df_train1.shape


(2617226, 69)

In [32]:
#Merged with repurchase
#
df_train2 = pd.merge(df_train1,df_repurchase,how='left',left_on='SCV_ID',right_on='SCV_ID')
df_train2.shape

(2617226, 71)

In [36]:
df_sep.shape

(2386161, 58)

In [30]:
#Retrieve flag with repurchase
df_train2.loc[(df_train2.TARGET_REPUR_IND ==1) & (df_train2.FLG_SUM >0)].shape[0]

14992

In [ ]:
#df_train1 = df_sep.merge(df_sep_flags, left_on='SCV_ID', right_on='SCV_ID',suffixes=('_pred', '_flag'))

In [27]:
# Merge cust360 and PRED
df_train3a = df_train2.merge(df_sep, left_on='SCV_ID', right_on='SCV_ID',suffixes=('_flag', '_pred'))
df_train3a.shape

(2347762, 127)

In [5]:
#df_train3 = df_train2.join(df_sep, on='SCV_ID', how='left', lsuffix='_flag', rsuffix='_pred')
#df_train3.shape

In [31]:
df_train3a.head()

REF_DT_flag     CUSTOMER_UNIQUE_ID     COMMON_IC        NEW_IC OLD_IC SERVICING_AGENT_CD    SCV_ID CUSTOMER_ROLE INFORCE_IND          ANN_INCOME MARITAL_STATUS          LIFE_STAGE                      OCCUPATION  NO_OF_CHILD VIT_MEMBERSHIP_STATUS VIT_STATUS_LAST_12M  VIT_AMOUNT_DUE VIT_AMOUNT_DUE_DATE  VIT_POINTS VIT_STATUS_CURRENT  VIT_POITNS_BAL VIT_BAL_DUE_DATE VIT_STATUS_BAL VIT_BAL_STATEMENT_IND FLG_REPURCHASE_YTD REPURCHASE_LIKELIHOOD LAPSE_LIKELIHOOD AD_CONV_LIKELIHOOD FLG_MARRIED_UPDATE_DT FLG_LIFE_STAGE_UPDATE_DT FLG_CHILD_UPDATE_DT FLG_INCOME_UPDATE_DT FLG_POL_ANNIVERSARY FLG_NON_ACUTE_CLM  FLG_NON_ACUTE_CLM_AMT FLG_ACUTE_CLM  FLG_ACUTE_CLM_AMT FLG_ACC_CLM  FLG_ACC_CLM_AMT FLG_NON_ACUTE_CLM_FAM  FLG_NON_ACUTE_CLM_AMT_FAM FLG_ACUTE_CLM_FAM  FLG_ACUTE_CLM_AMT_FAM FLG_ACC_CLM_FAM  FLG_ACC_CLM_AMT_FAM FLG_CALL_POSITIVE CALL_DATE CALL_TYPE FLG_CALL_NEGATIVE MYAIA_REGISTRATION_IND  ... INS_GENDER INS_MARITAL_STATUS      INS_LIFE_STAGE                   INS_OCCUPATON  \
0  2020-09-30  013187372455-19780831  013187372455  013187372455   None              4854Q  14939955          BOTH           Y    RM 36,001-72,000         Single       MATURE SINGLE   Trading (Office Work - Import            0                  None                None             NaN                 NaT         NaN               None             NaN              NaT           None                  None                  N                     -  Somewhat Likely                  -                   NaT                      NaT                 NaT                  NaT                   N                 N                  0.000             N              0.000           N            0.000                     N                      0.000                 N                  0.000               N                0.000                 N       NaT      None                 N                      N  ...          F             Single       Mature Single   Trading (Office Work - Import   
1  2020-09-30  013187372483-19780831  013187372483  013187372483   None              A7242   3407282            LF           Y    RM 36,001-72,000         Single       MATURE SINGLE  Senior Management, Manager, Ex            0                  None                None             NaN                 NaT         NaN               None             NaN              NaT           None                  None                  N                     -  Somewhat Likely                  -                   NaT                      NaT                 NaT                  NaT                   N                 N                  0.000             N              0.000           N            0.000                     N                      0.000                 N                  0.000               N                0.000                 N       NaT      None                 N                      Y  ...          F             Single       Mature Single  Senior Management, Manager, Ex   
2  2020-09-30  013187372574-19780831  013187372574  013187372574   None              1363A   2696849          BOTH           Y  RM 180,001-300,000        Married        NEST BUILDER                  Sales Director            1                  None                None             NaN                 NaT         NaN               None             NaN              NaT           None                  None                  N                     -  Somewhat Likely                  -                   NaT                      NaT                 NaT                  NaT                   Y                 N                  0.000             N              0.000           N            0.000                     N                      0.000                 N                  0.000               N                0.000                 N       NaT      None                 N                      N  ...          M            Married        Nest Builder                  Sales Director   
3  2020-09-30  013187372712-19780831  0131

In [55]:
##################

df_train4 = df_train3a[df_train3a.SCV_ID.notna()]
df_train4.shape

(2347762, 127)

In [56]:
df_train4.loc[(df_train4.TARGET_REPUR_IND ==1) & (df_train4.FLG_SUM >0)].shape[0]

14360

In [64]:
df_train4['REPURCHASE_ANP'] = df_train4['REPURCHASE_ANP'].fillna(0)
df_train4['TARGET_REPUR_IND'] = df_train4['TARGET_REPUR_IND'].fillna(0)

In [65]:
# Use columns that don't have missing values
df_train5 = df_train4[df_train4.columns[~df_train4.isnull().any()]]
df_train5.columns

Index(['REF_DT_flag', 'CUSTOMER_UNIQUE_ID', 'COMMON_IC', 'SERVICING_AGENT_CD', 'SCV_ID', 'CUSTOMER_ROLE', 'INFORCE_IND', 'NO_OF_CHILD', 'FLG_REPURCHASE_YTD', 'REPURCHASE_LIKELIHOOD', 'LAPSE_LIKELIHOOD', 'AD_CONV_LIKELIHOOD', 'FLG_POL_ANNIVERSARY', 'FLG_NON_ACUTE_CLM', 'FLG_NON_ACUTE_CLM_AMT', 'FLG_ACUTE_CLM', 'FLG_ACUTE_CLM_AMT', 'FLG_ACC_CLM', 'FLG_ACC_CLM_AMT', 'FLG_NON_ACUTE_CLM_FAM', 'FLG_NON_ACUTE_CLM_AMT_FAM', 'FLG_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_AMT_FAM', 'FLG_ACC_CLM_FAM', 'FLG_ACC_CLM_AMT_FAM', 'FLG_CALL_POSITIVE', 'FLG_CALL_NEGATIVE', 'MYAIA_REGISTRATION_IND', 'FHC_COMPLETION_IND', 'FLG_PPT', 'FLG_AGE_21', 'FLG_MARRIED', 'FLG_LIFE_STAGE', 'FLG_CHILD', 'FLG_INCOME_CHANGE', 'FLG_BIRTHDAY', 'FLG_FHC_ANNUAL', 'FLG_CLM_TRIGGER', 'FLG_CLM_FAM_TRIGGER', 'FLG_CALL_TRIGGER', 'FLG_MATURING', 'FLG_MATURED', 'FLG_SUM', 'REPURCHASE_ANP', 'TARGET_REPUR_IND', 'INS_ID', 'REF_DT_pred', 'INS_INCOME_SEGM', 'INS_GENDER', 'INS_NON_ACUTE', 'INS_CLM_MAJOR', 'INS_ACUTE_CT_6', 'INS_PAID_AMT_ACUTE_6',


In [67]:
df_train5 = pd.concat([df_train5,df_train4[['INS_L_AGT_SEGM','INS_AGE','INS_LIFE_STAGE']]],axis=1)

In [68]:
df_train5[['FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']].apply(pd.Series.value_counts).fillna(0)

FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0     2340607      2334494         2215514    2313669            2329885       2131095     2347762.000          2343632              2333364           2314955       2337730      2344058
1        7155        13268          132248      34093              17877        216667           0.000             4130                14398             32807         10032         3704

In [69]:
# Take data with at least 1 flag
df_train5 = df_train5[df_train5.FLG_SUM > 0]
df_train5.shape

(426915, 86)

In [70]:
df_train5.FLG_SUM.value_counts()

1    377122
2     41800
3      6477
4      1359
5       152
6         5
Name: FLG_SUM, dtype: int64

In [71]:
# drop FLG_BIRTHDAY with no repurchase
df_train6 = df_train5.drop(df_train5[(df_train5.FLG_SUM == 1) & (df_train5.FLG_BIRTHDAY == 1) &
                                     (df_train5.TARGET_REPUR_IND == 0)].index)
df_train6.shape

(239194, 86)

In [72]:
# # drop FLG_LIFE_STAGE with no repurchase
df_train7 = df_train6.drop(df_train6[(df_train6.FLG_SUM == 1) & (df_train6.FLG_LIFE_STAGE == 1) &
                                     (df_train6.TARGET_REPUR_IND == 0)].index)
df_train7.shape

(144734, 86)

In [73]:
df_train7[['FLG_AGE_21','FLG_MARRIED','FLG_LIFE_STAGE','FLG_CHILD','FLG_INCOME_CHANGE','FLG_BIRTHDAY','FLG_FHC_ANNUAL','FLG_CLM_TRIGGER',
'FLG_CLM_FAM_TRIGGER','FLG_CALL_TRIGGER','FLG_MATURING','FLG_MATURED']].apply(pd.Series.value_counts).fillna(0)

FLG_AGE_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_CHILD  FLG_INCOME_CHANGE  FLG_BIRTHDAY  FLG_FHC_ANNUAL  FLG_CLM_TRIGGER  FLG_CLM_FAM_TRIGGER  FLG_CALL_TRIGGER  FLG_MATURING  FLG_MATURED
0      137579       131466          106946     110641             126857        115788      144734.000           140604               130336            111927        134702       141030
1        7155        13268           37788      34093              17877         28946           0.000             4130                14398             32807         10032         3704

In [75]:
df_train7.to_csv(filepath + 'cust360_sep_cleaned.csv',index=False)